### 2022 College Women's Basketball LGBM 0223

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
paths=[]
names=[]
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths+=[os.path.join(dirname, filename)]
        names+=[filename[-4:]]

In [ ]:
for i in range(len(paths)):
    names[i]=pd.read_csv(paths[i],encoding='cp932')
    print(str(i)+'. '+paths[i].split('/')[-1][0:-4])
    #display(names[i])

In [ ]:
cresult1=names[3].copy()
cresult1p=cresult1[['Season','WTeamID','LTeamID']]
cresult2=names[8].copy()
cresult2p=cresult2[['Season','WTeamID','LTeamID']]
cresult=pd.concat([cresult1p,cresult2p],axis=0)
cresult=cresult.reset_index(drop=True)
cresult

In [ ]:
cresultA=cresult.copy()
cresultA.columns=['Season','Team1','Team2']
cresultA['Pred']=1
cresultA

In [ ]:
cresultB=cresult.copy()
cresultB.columns=['Season','Team2','Team1']
cresultB['Pred']=0
cresultB

In [ ]:
cresultC=pd.concat([cresultA,cresultB],axis=0).reset_index(drop=True)
train=cresultC

In [ ]:
names[8]

In [ ]:
test0=names[2].copy()
for i in range(len(test0)):
    IDs=test0.loc[i,'ID'].split('_')
    test0.loc[i,'Season']=int(IDs[0])
    test0.loc[i,'Team1']=int(IDs[1])
    test0.loc[i,'Team2']=int(IDs[2])
test=test0[['Season','Team1','Team2']].astype(int)
test

In [ ]:
import lightgbm as lgb
import random
import optuna
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [ ]:
target = train['Pred']
data = train.drop('Pred',axis=1)

In [ ]:
columns=data.columns.to_list()
print(columns)

In [ ]:
def objective(trial,data=data,target=target):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param =   {
        'num_leaves': trial.suggest_int('num_leaves', 100,300),
        'objective': trial.suggest_categorical('objective',['rmse','binary']),  
        'max_depth': -1,
        'learning_rate': 0.1,
        "boosting": "gbdt",
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        "bagging_freq": 5,
        "bagging_fraction": trial.suggest_uniform('bagging_fraction', 0.6, 1.0),
        "feature_fraction": trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        "verbosity": -1,
    }
    model = lgb.LGBMRegressor(**param)      
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    preds = model.predict(test_x)
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
# shows the scores from all trials
optuna.visualization.plot_optimization_history(study)

In [ ]:
# interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# shows the evolution of the search
optuna.visualization.plot_slice(study)

In [ ]:
# Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
# Visualize empirical distribution function
optuna.visualization.plot_edf(study)

In [ ]:
Best_trial=study.best_trial.params
print(Best_trial)

In [ ]:
print(test.shape)

In [ ]:
preds = np.zeros((test.shape[0]))
kf = KFold(n_splits=5,random_state=48,shuffle=True)
for trn_idx, test_idx in kf.split(train[columns],target):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=target.iloc[trn_idx],target.iloc[test_idx]
    model = lgb.LGBMRegressor(**Best_trial)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds+=model.predict(test[columns])/kf.n_splits   ###### predict_proba
    rmse=mean_squared_error(y_val, model.predict(X_val),squared=False)
    print(rmse)

In [ ]:
subm=names[2].copy()
subm['Pred'] = preds
subm.to_csv('submission.csv',index=False)
subm